<a href="https://colab.research.google.com/github/hahajjjun/Disastogram_Project/blob/main/disastogram_text_auto_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Basic Settings

In [1]:
import pandas as pd

In [2]:
path = "/content/drive/MyDrive/NLP_Project/안전안내문자_완성본.csv"
df = pd.read_csv(path)

In [3]:
df = df.drop(columns = ['Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8'])

In [4]:
df.columns = ['장소', '텍스트', 'Class', 'Label', '비고']

In [5]:
df_covid = df[df['Label']==1]

In [6]:
df_covid.Class = df_covid.Class.fillna(0)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [7]:
df_covid['Class'] = df_covid['Class'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
df_covid['Label'] = df_covid['Class'].astype(bool)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
df_covid_1 = df_covid[df_covid['Class']==1]
df_covid_2 = df_covid[df_covid['Class']==2]
df_covid_3 = df_covid[df_covid['Class']==3]
df_covid_4 = df_covid[df_covid['Class']==4]
df_covid_5 = df_covid[df_covid['Class']==5]
df_covid_6 = df_covid[df_covid['Class']==6]
df_covid_7 = df_covid[df_covid['Class']==7]
print(len(df_covid), len(df_covid_1), len(df_covid_2), len(df_covid_3), len(df_covid_4), len(df_covid_5), len(df_covid_6), len(df_covid_7))

1649 86 266 444 207 581 57 1


##Class 1 Embedding

In [78]:
total = []
for i in range(len(df_covid_1)):
    keywords = []
    # 1st keyword
    flag = True
    query_1 = ['외국인', '근로자']
    query_2 = ['해외', '입국']
    query_3 = ['유증상', '의심', '증상']
    if flag:
        for item_1 in query_1:
            if item_1 in df_covid_1['텍스트'].iloc[i] and flag:
                keywords.append('외국인 근로자')
                flag = False

    if flag:
        for item_2 in query_2:
            if item_2 in df_covid_1['텍스트'].iloc[i] and flag :
                keywords.append('해외입국자')
                flag = False
    
    if flag:
        keywords.append('유증상자')
    # 2nd keyword
    flag = True
    suffix = '보건소'
    query_4 = ['시','도','군','구'] 
    # 시흥처럼 지역명 안에 '시, 도, 군, 구'가 들어가는 경우 고려
    for item_4 in query_4:
        location = df_covid_1['장소'].iloc[i]
        text = df_covid_1['텍스트'].iloc[i]
        if '재난안전' in text:
            suffix = '재난안전대책본부'
        if '시청' in text:
            suffix = '청'
        if (location.find(item_4) != -1 and flag):
            flag = False
            pos = location.find(item_4)
            keywords.append(location[0:pos]+item_4+suffix)
    
    # 3rd keyword
    flag = True
    text = df_covid_1['텍스트'].iloc[i]
    if '검사' in text:
        keywords.append('검사')
    else:
        keywords.append('신고')
    total.append(keywords)
df_covid_1['embedding']=total
df_covid_1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,장소,텍스트,Class,Label,비고,embedding
0,군산시청,최근 2주 이내 외국에 다녀오신 분들은 군산시 보건소(063-463-4000)로 ...,1,True,NaN,"[유증상자, 군산시보건소, 신고]"
7,창녕군청,"코로나19 예방을 위해 3월22일 이후 모든 해외입국자 및 입국예정 본인, 가족께서...",1,True,NaN,"[해외입국자, 창녕군보건소, 신고]"
12,서초구청,오늘부터 모든 해외입국자는 2주간 자가격리 의무대상자입니다. 증상유무와 상관없이 서...,1,True,NaN,"[해외입국자, 서초구보건소, 검사]"
16,서울시청,"서울시민 중 ""3.13.이후 해외입국자""는 유증상시 보건소 방문 코로나19 검사 ...",1,True,NaN,"[해외입국자, 서울시보건소, 검사]"
27,아산시청,"3월14일 이후 유럽, 미국에서 입국하신 분은 아산시 보건소(041-536-853...",1,True,NaN,"[해외입국자, 아산시보건소, 신고]"
...,...,...,...,...,...,...
4098,대구시,외국인 근로자에 대해 강제출국 걱정없이 코로나19검사를 무료로 실시(체류자격 무관...,1,True,NaN,"[외국인 근로자, 대구시보건소, 검사]"
4200,정선군청,"8.15 광복절집회, 8.7~8.15 서울사랑제일교회 방문자는 증상유무 관계없이 정...",1,True,NaN,"[유증상자, 정선군보건소, 검사]"
4206,의정부시청,의정부시 43번 확진자 추가동선 관련 6.17~6.21 의정부시 범골로 유흥주점 ...,1,True,NaN,"[유증상자, 의정부시보건소, 신고]"
4262,전라북도청1,"대전50번,55번 확진자 6월12일 17:15~17:44 전주 청년다방 도청점 방문...",1,True,NaN,"[유증상자, 전라북도보건소, 신고]"


## Class 2 Embedding

In [112]:
total = []
DB = ['타워', '병원', '센터', '클럽', '마트', '카페', '버스', '온천', '저수지', '사무소', '다방', '사우나', '가게', '식당', '약국', '연습장', '회관', '식장', '스파', '집회']
Rem = ['에', '를', '을', '가', '는', '이', '은', ',']
people_1 = ['방문', '참석', '이용', '참가', '탑승']
people_2 = ['미검사', '증상']
people_3 = ['근로']

for i in range(len(df_covid_2)):
    # keyword 1
    flag = True
    key = ""
    keywords = []
    sentence = df_covid_2['텍스트'].iloc[i]
    words = sentence.split()
    for loc in DB:
        for word in words:
            if loc in word and flag:
                key = word
                pos = key.find(loc)
                flag = False
    if key != "":
        for r in Rem:
            suffix_pos = key.find(r)
            if suffix_pos != -1:
                if(suffix_pos > pos):
                    key = key[0:suffix_pos]+key[suffix_pos+1:]
        if '(' in key and ')' not in key:
            key = key+')'                
        keywords.append(key)
    #keyword 2
    flag = True
    for item_2 in people_2:
        if item_2 in sentence and flag:
            keywords.append('유증상자')
            flag = False

    for item_1 in people_1:
        if item_1 in sentence and flag:
            keywords.append('방문자')
            flag = False
    
    for item_3 in people_3:
        if item_3 in sentence and flag:
            keywords.append('근로자')
            flag = False
    #keyword 3
    flag = True
    if '검사' in sentence:
        keywords.append('검사')
    else:
        keywords.append('신고')
    total.append(keywords)
df_covid_2['embedding']=total
df_covid_2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,장소,텍스트,Class,Label,비고,embedding
17,함안군청,"진주윙스타워가 지역감염원으로 추정, 군민방문자제 및 3월25일 이후 진주 윙스타워를...",2,True,NaN,"[진주윙스타워, 방문자, 신고]"
57,의령군청,"진주 윙스타워(3월12일 이후), 진주 스파랜드(3월25일 이후)를 방문하신 군민은...",2,True,NaN,"[윙스타워(3월12일), 방문자, 신고]"
64,남해군청,"진주 확진자 발생관련, 진주 윙스타워(3월15일 이후)를 방문하신 주민은 남해군 보...",2,True,NaN,"[윙스타워(3월15일), 방문자, 신고]"
75,울진군청,"2주만 잠시 멈춤, 사회적 거리두기(2m 이상, 봄나들이 등 외출 자제)로 청정 울...",2,True,NaN,[신고]
82,연천군청,"3.17이후 의정부성모병원 방문자(병문안,종사자,면회객,간병인)는 연천군 선별진료소...",2,True,NaN,"[의정부성모병원, 방문자, 검사]"
...,...,...,...,...,...,...
4395,전남도청,(동방고속 74바5004) 탑승자는 진단검사 받으세요 5.11(화)▲8:38순천→...,2,True,NaN,"[방문자, 검사]"
4417,경남도청,4.1. ~ 4.4. 창원 컨벤션센터(의창구 원이대로 362) 베이비페어 현대해상...,2,True,NaN,"[컨벤션센터(의창구), 유증상자, 신고]"
4471,중대본,"11.26∼12.4 이태원소재 웨스턴라운지,투페어,다이스,젠틀레빗,KMGM업소 방문...",2,True,NaN,"[방문자, 검사]"
4474,연수구청,"11.13(금),14(토)15:00~24:00,11.15(일)15:00~16:40연...",2,True,NaN,"[방문자, 검사]"


## Class 3 Embedding

In [132]:
queries = [['단계'],['사회적', '거리', '위생', '수칙', '자제', '금지'],['해외', '입국'], ['방역'], ['폐쇄'], ['전자']]
key = ['거리두기 정책', '일반 방역 수칙', '해외 입국', '방역', '폐쇄 조치', '전자출입명부', '행정명령']
subclass_total = []
for i in range(len(df_covid_3)):
    flag = True
    subclass = ""
    sentence = df_covid_3['텍스트'].iloc[i]
    for j in range(len(queries)):
        for item in queries[j]:
            if item in sentence and flag:
                subclass = key[j]
                flag = False
    if subclass == "":
        subclass = key[1]
    subclass_total.append(subclass)
df_covid_3['세부 분류']=subclass_total

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [136]:
total = []
key = ['거리두기 정책', '일반 방역 수칙', '해외 입국', '방역','폐쇄 조치', '전자출입명부', '행정명령']
for i in range(len(df_covid_3)):
    sentence = df_covid_3['텍스트'].iloc[i]
    location = df_covid_3['장소'].iloc[i]
    subclass = df_covid_3['세부 분류'].iloc[i]
    flag = True
    if key.index(subclass) == 0: #거리두기 정책
        for word in sentence:
            for char in word:
                if char.isdigit() and flag:
                    keyword = ['거리두기'+char+'단계']
                    flag = False

    elif key.index(subclass) == 1: #일반 방역수칙
        keyword = ['사회적 거리두기 및 방역수칙 준수']

    elif key.index(subclass) == 2: #해외입국
        keyword = ['해외 입국']
        flag = True
        suffix = '보건소'
        loc_query = ['시','도','군','구'] 
        # 시흥처럼 지역명 안에 '시, 도, 군, 구'가 들어가는 경우 고려
        for item in loc_query:
            if '재난안전' in sentence:
                suffix = '재난안전대책본부'
            if '시청' in sentence:
                suffix = '청'
            if (location.find(item) != -1 and flag):
                flag = False
                pos = location.find(item)
                keyword.append(location[0:pos]+item+suffix)
        if '검사' in sentence:
            keyword.append('검사')
        else:
            keyword.append('신고')

    elif key.index(subclass) == 3: #방역
        keyword = []
        flag = True
        loc_query = ['시','도','군','구'] 
        # 시흥처럼 지역명 안에 '시, 도, 군, 구'가 들어가는 경우 고려
        for item in loc_query:
            if (location.find(item) != -1 and flag):
                flag = False
                pos = location.find(item)
                keyword.append(location[0:pos]+item+'방역 실시')

    elif key.index(subclass) == 4: #폐쇄 조치
        for word in sentence:
            if '폐쇄' in word:
                keyword = [word]

    elif key.index(subclass) == 5: #전자출입명부
        keyword = ['전자출입명부 사용']

    elif key.index(subclass) == 6: #행정명령
        keyword = []
    total.append(keyword)
df_covid_3['embedding']=total
df_covid_3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,장소,텍스트,Class,Label,비고,세부 분류,embedding
1,부산광역시,"타지역보다 벚꽃이 빨리피는 부산, 코로나19 확산방지를 위해 꽃놀이를 자제해 주시...",3,True,NaN,일반 방역 수칙,[사회적 거리두기 및 방역수칙 준수]
2,통영시청,인근지역 확진자 발생에 따른 코로나 예방 및 서로의 안전을 위하여 사회적 거리두기...,3,True,NaN,일반 방역 수칙,[사회적 거리두기 및 방역수칙 준수]
8,대구,"코로나19 젊은 사람들도 절대조심. 마스크 착용은 필수, 클럽·PC방 등 밀폐된 장...",3,True,NaN,일반 방역 수칙,[사회적 거리두기 및 방역수칙 준수]
15,안양시청,3월25일 이후 입국하신 시민분들은 증상유무와 관계없이 검사대상이오니 보건소 선별진...,3,True,보건소 연락이지만 방역수칙과 관계 있음,해외 입국,"[해외 입국, 안양시보건소, 검사]"
19,여수시청,"3.1.이후 가족중 해외 입국예정자는 필히 사전상담(061-659-4252), 2주...",3,True,NaN,해외 입국,"[해외 입국, 여수시보건소, 신고]"
...,...,...,...,...,...,...,...
4468,충남도청,충남 사회적거리두기 2단계 조치가 연장됩니다.(당진시 2.5단계) ▲기간: 12.1...,3,True,NaN,거리두기 정책,[거리두기2단계]
4472,충남도청,12월 1일부터 충남도 거리두기 1.5단계 격상(천안은 2단계)에 따라 집단감염 고...,3,True,NaN,거리두기 정책,[거리두기1단계]
4479,충청남도청,"추석연휴 동안 고향이나 여행지 방문자 중 발열,기침 등 증상이 있을 경우 가까운 보...",3,True,NaN,일반 방역 수칙,[사회적 거리두기 및 방역수칙 준수]
4480,충남도청,"""이번 추석 연휴는 충남도민 잠시 멈춤"" 감염병 예방을 위해 외출·모임·여행 자제...",3,True,NaN,일반 방역 수칙,[사회적 거리두기 및 방역수칙 준수]


## Class 4 Embedding

In [139]:
queries = [['해외', '입국', '외국', '근로', '방문'], ['보건', '선별', '진료'],['마스크'],['긴급', '재난지원','기본','소득','지원금'],['군인','백신','0','1','2','3','4','5','6','7','8','9'], ['폐쇄','방역','요금'], ['음성']]
keys = [1,2,0,3,4,5,7]
subclass_total = []
for i in range(len(df_covid_4)):
    flag = True
    subclass = ""
    sentence = df_covid_4['텍스트'].iloc[i]
    for j in range(len(queries)):
        for item in queries[j]:
            if item in sentence and flag:
                subclass = keys[j]
                flag = False
    if subclass == "":
        subclass = keys[6]
    subclass_total.append(subclass)
df_covid_4['세부 분류']=subclass_total

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [147]:
total = []
for i in range(len(df_covid_4)):
    sentence = df_covid_4['텍스트'].iloc[i]
    location = df_covid_4['장소'].iloc[i]
    subclass = df_covid_4['세부 분류'].iloc[i]
    flag = True
    if subclass == 0: #마스크 배부
        keyword = ['공적마스크', '배부']

    elif subclass == 1: #보건소 방문
        keyword = []
        flag = True
        suffix = '보건소'
        loc_query = ['시','도','군','구'] 
        # 시흥처럼 지역명 안에 '시, 도, 군, 구'가 들어가는 경우 고려
        for item in loc_query:
            if '재난안전' in sentence:
                suffix = '재난안전대책본부'
            if '시청' in sentence:
                suffix = '청'
            if (location.find(item) != -1 and flag):
                flag = False
                pos = location.find(item)
                keyword.append(location[0:pos]+item+suffix)
        if '검사' in sentence:
            keyword.append('검사')
        else:
            keyword.append('신고')

    elif subclass == 2: #보건소 운영
        keyword = []
        flag = True
        suffix = '보건소'
        loc_query = ['시','도','군','구'] 
        # 시흥처럼 지역명 안에 '시, 도, 군, 구'가 들어가는 경우 고려
        for item in loc_query:
            if '재난안전' in sentence:
                suffix = '재난안전대책본부'
            if '시청' in sentence:
                suffix = '청'
            if (location.find(item) != -1 and flag):
                flag = False
                pos = location.find(item)
                keyword.append(location[0:pos]+item+suffix)
        if '중단' in sentence:
            keyword.append('다른 보건소 방문')
        else:
            keyword.append('보건소 운영 중')

    elif subclass == 3: #긴급재난지원
        keyword = ['재난기본소득']
        if '온라인' in sentence:
            keyword.append('온라인 신청')
        elif '우편' in sentence:
            keyword.append('우편 신청')
        elif '방문' in sentence:
            keyword.append('방문 신청')
        if '마감' in sentence:
            keyword.append('마감 예정')

    elif subclass == 4: #백신 예약
        keyword = ['백신', '사전 예약 및 신청']

    elif subclass == 5: #특정 장소
        if '폐쇄' in sentence :
            keyword = ['폐쇄']

    elif subclass == 7: #음성 판정
        temp = ''
        words = sentence.split()
        for word in words:
            for char in word:
                if char.isdigit():
                    temp = temp+char
        keyword = ['확진자'+temp]
        if '음성' in sentence : 
            keyword = ['음성 판정']
        
    total.append(keyword)
df_covid_4['embedding']=total
df_covid_4

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,장소,텍스트,Class,Label,비고,세부 분류,embedding
3,사천시청,"해외입국자 인천공항에서 사천까지 교통편의 제공, 가족분은 보건소(055-831-36...",4,True,보건소 연락이지만 방역수칙과는 무관,1,"[사천시보건소, 신고]"
6,괴산군청,긴급재난지원금을 5.18일부터 출생년도 끝자리 요일별로 읍면 및 군홈페이지를 통해...,4,True,NaN,3,[재난기본소득]
20,양평군청,3월25일부터 해외입국 군민대상 코로나19 검사 무료. 방문전 상담(031-770-...,4,True,NaN,1,"[양평군보건소, 검사]"
24,공주시청,"<코로나19 무료검사> 해외입국자(3.13~3.26유럽,미국입국자/3.27이후 모든...",4,True,NaN,1,"[공주시보건소, 검사]"
40,거창군보건소,"약국 공적마스크 판매시간 안내(3/29) 9시~ 태평양,백두산,시민 10시~ 건강...",4,True,NaN,0,"[공적마스크, 배부]"
...,...,...,...,...,...,...,...
4378,달서구청,60~74세(47~61년생)백신접종 6/3까지 예약. 온라인(대리가능) ncvr....,4,True,NaN,4,"[백신, 사전 예약 및 신청]"
4413,대덕구청,코로나19 확진자 집단발생으로 관내 도시공원을 전면폐쇄(6.20.~7.5.) 하오...,4,True,NaN,4,"[백신, 사전 예약 및 신청]"
4427,경기도청,'제2차 경기도 재난기본소득'을 신청하지 않은 경기도민은 지금 홈페이지(basic...,4,True,NaN,3,[재난기본소득]
4430,경기도청,'제2차 경기도 재난기본소득'을 신청하지 않은 경기도민은 지금 홈페이지(basic...,4,True,NaN,3,[재난기본소득]


## Class 5 Embedding

In [161]:
total = []
for i in range(len(df_covid_5)):
    sentence = df_covid_5['텍스트'].iloc[i]
    query_1 = ['명']
    query_2 = ['번째','번']
    flag = True
    keyword = []
    words = sentence.split()
    for word in words:
        for item in query_1:
            if item in word and flag:
                temp =''
                state = True
                for j in range(len(word)):
                    if state == False: #7번째확진자30세 => 730번째확진자되는 문제해결
                        if word[j].isdigit() and flag:
                            temp = temp + word[j]
                        else:
                            flag = False
                    else:
                        if word[j].isdigit() and flag:
                            temp= temp + word[j]
                            state = False
                if temp != '':
                    keyword=['확진자'+temp+'명']
                
        for item in query_2:
            if item in word and flag:
                temp =''
                state = True
                for j in range(len(word)):
                    if state == False:
                        if word[j].isdigit() and flag:
                            temp = temp + word[j]
                        else:
                            flag = False
                    else:
                        if word[j].isdigit() and flag:
                            temp= temp + word[j]
                            state = False
                if temp != '':
                    keyword=[temp+item+'확진자']
                flag = False
    
    query_3 = ['진행','역학','조사']
    query_4 = ['동선','경로']

    flag = True
    for item in query_3:
        if item in sentence and flag:
            keyword.append('역학조사')
            flag = False
    for item in query_4:
        if item in sentence and flag:
            keyword.append('동선공개')
    
    total.append(keyword)
df_covid_5['embedding']=total
df_covid_5

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,장소,텍스트,Class,Label,비고,embedding
4,오산시,"코로나19, 5번째 확진자 발생(남, 34세, 오산동, 해외입국자)역학조사 후 오산...",5,True,NaN,"[5번째확진자, 역학조사]"
5,거제시청,7번확진자발생(남/뉴질랜드/30세/양정) 4.8(수)09:30양성판정/해외여행력(미...,5,True,NaN,"[7번확진자, 역학조사]"
9,양천구청,"양천구 확진자 발생(17번, 미국 귀국) 및 동선안내. 양천구 블로그 참조 blo...",5,True,NaN,"[17번확진자, 동선공개]"
10,시흥시,"확진환자 1명 발생(66년생, 여, 정왕2동 거주, 금천구 가산콜센터 근무)거주지 ...",5,True,NaN,"[확진자1명, 역학조사]"
11,철원군청,"3월31일(화) 코로나19 확진자 발생 (60대, 여, 동송읍, 의정부성모병원 근...",5,True,NaN,[]
...,...,...,...,...,...,...
4351,서대문구청,"코로나19. 36번째 확진자(홍은2동, 해외입국자)발생. 자세한 사항 구홈페이지 및...",5,True,NaN,[36번째확진자]
4357,관악구청,"84번째 확진자 발생(은천동), 82번째 및 타 지역 확진자 동선 공개, 자세한내용...",5,True,NaN,"[84번째확진자, 동선공개]"
4382,중구청,"대전151번∼155번=중구14번∼18번 접촉자151명 중(음성42,검사중109),...",5,True,NaN,"[151번확진자, 동선공개]"
4390,홍천군청,"성남시 No.184,185 코로나19 확진자 홍천군 00캠핑장 방문관련 이동경로를 ...",5,True,NaN,[동선공개]


## Output

In [162]:
df_covid.to_excel(r'/content/drive/MyDrive/NLP_Project/covid_sudo.xlsx', header=False)
df_covid_1.to_excel(r'/content/drive/MyDrive/NLP_Project/covid_1_sudo.xlsx',header=False)
df_covid_2.to_excel(r'/content/drive/MyDrive/NLP_Project/covid_2_sudo.xlsx',header=False)
df_covid_3.to_excel(r'/content/drive/MyDrive/NLP_Project/covid_3_sudo.xlsx', header=False)
df_covid_4.to_excel(r'/content/drive/MyDrive/NLP_Project/covid_4_sudo.xlsx',header=False)
df_covid_5.to_excel(r'/content/drive/MyDrive/NLP_Project/covid_5_sudo.xlsx',header=False)
df_covid_6.to_excel(r'/content/drive/MyDrive/NLP_Project/covid_6_sudo.xlsx',header=False)
df_covid_7.to_excel(r'/content/drive/MyDrive/NLP_Project/covid_7_sudo.xlsx',header=False)